In [35]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import string

from keras.models import Model
from keras.layers import Input, LSTM, Dense

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/englishhindi-translation-dataset-iit-bombay/hindi_english_parallel.csv


## 1. Loading data 

In [6]:
df = pd.read_csv("/kaggle/input/englishhindi-translation-dataset-iit-bombay/hindi_english_parallel.csv")

In [7]:
df.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [8]:
ndf = df.dropna(axis=0)

In [9]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1555727 entries, 0 to 1561839
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   hindi    1555727 non-null  object
 1   english  1555727 non-null  object
dtypes: object(2)
memory usage: 35.6+ MB


In [10]:
ndf.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [11]:
ndf.iloc[0,:]

hindi      अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
english     Give your application an accessibility workout
Name: 0, dtype: object

In [28]:
ndf['hindi']

0            अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1                            एक्सेर्साइसर पहुंचनीयता अन्वेषक
2                      निचले पटल के लिए डिफोल्ट प्लग-इन खाका
3                       ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
4          उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...
                                 ...                        
1561835                      Members making oath/affirmation
1561836    स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...
1561837    मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...
1561838    उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...
1561839    खाद्य और जल सुरक्षा; पर्यावरण की दृष्टि से वहन...
Name: hindi, Length: 1555727, dtype: object

## 2. Tokenizing the data

In [43]:
input_char = set()
target_char = set()

# total unique words in hindi corpus
for line in ndf['hindi'].values:
    for char in line:
        if char not in input_char:
            if char not in string.punctuation:
                input_char.add(char)
        
# total unique words in english corpus
for line2 in ndf['english'].values:
    for char2 in line2:
        if char2 not in target_char:
            if char2 not in string.punctuation:
                target_char.add(char2)

In [50]:
# find chars to encode and max encoder seq length
input_characters = sorted(list(input_char))
target_characters = sorted(list(target_char))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in ndf['hindi'].values])
max_decoder_seq_length = max([len(txt) for txt in ndf['english'].values])

print('Number of samples:', len(ndf['hindi']))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1555727
Number of unique input tokens: 476
Number of unique output tokens: 460
Max sequence length for inputs: 8000
Max sequence length for outputs: 11088


In [54]:
# indexing the input and target tokens
input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i, char in enumerate(target_characters)])

In [57]:
# encoder and decoder input data 
encoder_input_data = np.zeros(
    (len(ndf), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(ndf), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(ndf), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')